In [ ]:
import zipfile
with zipfile.ZipFile("cricket-bat-detection-new.zip","r") as f:
  f.extractall()

In [12]:
# Import necessary modules
import ast  # Import the 'ast' module for abstract syntax tree parsing
import os  # Import the 'os' module for file and directory operations
from xml.dom import minidom  # Import 'minidom' from 'xml.dom' for XML parsing

# Create a directory named "out" if it doesn't exist
os.makedirs("out", exist_ok=True)

# Parse the XML annotation file (replace with the actual file path)
file = minidom.parse('annotations (3) (2).xml')

# Get all 'image' elements from the XML file
images = file.getElementsByTagName('image')

# Loop through each 'image' element in the XML file
for image in images:
    try:
        # Extract width, height, and name attributes from the 'image' element
        width = int(image.getAttribute('width'))
        height = int(image.getAttribute('height'))
        name = image.getAttribute('name')

        # Get 'points' and 'box' elements
        elem = image.getElementsByTagName('points')
        bbox = image.getElementsByTagName('box')[0]

        # Extract coordinates for bounding box
        xtl = int(float(bbox.getAttribute('xtl')))
        ytl = int(float(bbox.getAttribute('ytl')))
        xbr = int(float(bbox.getAttribute('xbr')))
        ybr = int(float(bbox.getAttribute('ybr')))

        # Calculate width and height of the bounding box
        w = xbr - xtl
        h = ybr - ytl

        # Create a label file for writing
        label_file = open(os.path.join("out", name[:-4] + '.txt'), 'w')

        # Loop through each 'points' element
        for e in elem:
            # Write bounding box information to the label file
            label_file.write('0 {} {} {} {} '.format(str((xtl + (w / 2)) / width), str((ytl + (h / 2)) / height),
                                                      str(w / width), str(h / height)))

            # Extract and parse the 'points' attribute
            points = e.attributes['points']
            points = points.value.split(';')
            points_ = []

            # Parse and append points to the list
            for p in points:
                p = p.split(',')
                p1, p2 = p
                points_.append([int(float(p1)), int(float(p2))])

            # Write point coordinates to the label file
            for p_, p in enumerate(points_):
                label_file.write('{} {}'.format(p[0] / width, p[1] / height))
                if p_ < len(points_) - 1:
                    label_file.write(' ')
                else:
                    label_file.write('\n')
    except:
        pass


In [2]:
import glob
len(glob.glob("out/*.txt"))

137

In [3]:
for fn in glob.glob("out/*.txt"):
  with open(fn) as f:
    if len(f.read().split())<13 or len(f.read().split())>13 :
      os.remove(fn)
      # print(fn)

In [7]:
img_dir = "/content/drive/MyDrive/cricket-bat-detection-new/cricket-images-new/" ## replace with data dir
lab_dir = "out/"

In [4]:
all_txts = glob.glob("out/*.txt")

In [ ]:
import os
os.makedirs("datasets/data/images/train")
os.makedirs("datasets/data/images/val")
os.makedirs("datasets/data/labels/train")
os.makedirs("datasets/data/labels/val")

In [5]:
train_txt = all_txts[:120]
val_txt = all_txts[120:]

In [ ]:
import shutil

In [ ]:
os.path.splitext("demo.txt")[0]

'demo'

In [ ]:
[
    (
        shutil.copy(txt,"datasets/data/labels/train"),
        shutil.copy(os.path.join(
            img_dir,
           f"{os.path.splitext(os.path.split(txt)[-1])[0]}.jpg"
            ),"datasets/data/images/train")

    )
    for txt in train_txt
]

In [ ]:
[
    (
        shutil.copy(txt,"datasets/data/labels/val/"),
        shutil.copy(os.path.join(
            img_dir,
           f"{os.path.splitext(os.path.split(txt)[-1])[0]}.jpg"
            ),"datasets/data/images/val")

    )
    for txt in val_txt
]

In [ ]:
# for dir in ["datasets/data/images/val","datasets/data/images/train"]:
#   for fn in os.listdir(dir):
#     shutil.copy(os.path.join(dir,fn),img_dir)

In [ ]:
!pip install ultralytics -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.0/617.0 kB 6.9 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO

fliplr = 0.0
model = YOLO('yolov8l-pose.pt')

model.train(data='config.yaml', epochs=20,fliplr = 0.0)